# TensorFlow!

In [1]:
import tensorflow
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers
import tensorflow.keras.utils as utils
import tensorflow.keras.optimizers as optimizers

client1_model = tensorflow.keras.models.load_model(
    'model_weight/global_model.h5')
client2_model = tensorflow.keras.models.load_model(
    'model_weight/global_model.h5')


# It's training time!

In [2]:
import numpy
import tensorflow.keras.callbacks as callbacks


def get_dataset():
	container = numpy.load('dataset.npz') # Download dataset.npz from the link in Readme.md
	b, v = container['b'], container['v']
	v = numpy.asarray(v / abs(v).max() / 2 + 0.5, dtype=numpy.float32) # normalization (0 - 1)
	return b, v

x_train, y_train = get_dataset()

In [3]:
# Split dataset for two clients
# pip install -U scikit-learn
from sklearn.model_selection import train_test_split

client1_x, client2_x, client1_y, client2y =train_test_split(x_train, y_train, test_size=0.5, random_state=100)

In [4]:
"""
Training client 1 model
"""
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint_filepath = '/checkpoint/f1-1/'
model_checkpointing_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
)
client1_model.fit(client1_x, client1_y,
          batch_size=2048,
          epochs=100,
          verbose=1,
          validation_split=0.1,
          callbacks=[callbacks.ReduceLROnPlateau(monitor='loss', patience=10),
                     callbacks.EarlyStopping(monitor='loss', patience=15, min_delta=1e-4), model_checkpointing_callback])


Epoch 1/100
330/330 [==============================] - ETA: 0s - loss: 2.8261e-04

INFO:tensorflow:Assets written to: /checkpoint/f1-1\assets


INFO:tensorflow:Assets written to: /checkpoint/f1-1\assets


330/330 [==============================] - 108s 325ms/step - loss: 2.8261e-04 - val_loss: 2.9088e-04 - lr: 5.0000e-06
Epoch 2/100
330/330 [==============================] - 105s 317ms/step - loss: 2.8215e-04 - val_loss: 2.9099e-04 - lr: 5.0000e-06
Epoch 3/100
330/330 [==============================] - 101s 306ms/step - loss: 2.8172e-04 - val_loss: 2.9123e-04 - lr: 5.0000e-06
Epoch 4/100
330/330 [==============================] - 105s 319ms/step - loss: 2.8139e-04 - val_loss: 2.9156e-04 - lr: 5.0000e-06
Epoch 5/100
330/330 [==============================] - 102s 310ms/step - loss: 2.8109e-04 - val_loss: 2.9157e-04 - lr: 5.0000e-06
Epoch 6/100
330/330 [==============================] - 100s 303ms/step - loss: 2.8081e-04 - val_loss: 2.9187e-04 - lr: 5.0000e-06
Epoch 7/100
330/330 [==============================] - 100s 302ms/step - loss: 2.8047e-04 - val_loss: 2.9209e-04 - lr: 5.0000e-06
Epoch 8/100
330/330 [==============================] - 100s 303ms/step - loss: 2.8022e-04 - val_loss: 

In [5]:
"""
Training client 2 model
"""
checkpoint_filepath = '/tmp/checkpoint/f1-2/'
model_checkpointing_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
)
client2_model.fit(client2_x, client2y,
          batch_size=2048,
          epochs=100,
          verbose=1,
          validation_split=0.1,
          callbacks=[callbacks.ReduceLROnPlateau(monitor='loss', patience=10),
                     callbacks.EarlyStopping(monitor='loss', patience=15, min_delta=1e-4), model_checkpointing_callback])


Epoch 1/100
330/330 [==============================] - ETA: 0s - loss: 2.8484e-04

INFO:tensorflow:Assets written to: /tmp/checkpoint/f1-2\assets


INFO:tensorflow:Assets written to: /tmp/checkpoint/f1-2\assets


330/330 [==============================] - 107s 322ms/step - loss: 2.8484e-04 - val_loss: 2.8831e-04 - lr: 5.0000e-06
Epoch 2/100
330/330 [==============================] - 103s 311ms/step - loss: 2.8433e-04 - val_loss: 2.8879e-04 - lr: 5.0000e-06
Epoch 3/100
330/330 [==============================] - 99s 301ms/step - loss: 2.8400e-04 - val_loss: 2.8854e-04 - lr: 5.0000e-06
Epoch 4/100
330/330 [==============================] - 100s 303ms/step - loss: 2.8369e-04 - val_loss: 2.8908e-04 - lr: 5.0000e-06
Epoch 5/100
330/330 [==============================] - 103s 313ms/step - loss: 2.8333e-04 - val_loss: 2.8904e-04 - lr: 5.0000e-06
Epoch 6/100
330/330 [==============================] - 106s 322ms/step - loss: 2.8305e-04 - val_loss: 2.8921e-04 - lr: 5.0000e-06
Epoch 7/100
330/330 [==============================] - 106s 321ms/step - loss: 2.8276e-04 - val_loss: 2.8937e-04 - lr: 5.0000e-06
Epoch 8/100
330/330 [==============================] - 106s 322ms/step - loss: 2.8252e-04 - val_loss: 2

In [6]:
"""
Aggregrate models
"""

client1_weights = client1_model.get_weights()
client2_weights = client2_model.get_weights()

client1_weights = numpy.array(client1_weights, dtype=object)
client2_weights = numpy.array(client2_weights, dtype=object)

client1_weights = 1/2 * client1_weights
client2_weights = 1/2 * client2_weights

aggregated_weights = numpy.add(client1_weights,client2_weights)

client1_model.set_weights(aggregated_weights)
client1_model.save_weights("model_weight/federated1_weights", save_format="h5")